In [1]:
!pip install transformers
!pip install pickle5

In [2]:
'''
1. order train_ds by length (sort of done)
2. iteriamo sul dataloader facendo padding mano a mano (vedi codice vecchio notebook)
'''


'\n1. order train_ds by length (sort of done)\n2. iteriamo sul dataloader facendo padding mano a mano (vedi codice vecchio notebook)\n'

In [3]:
import torch
import math
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchtext import data
import numpy as np
import dataset
import net
import params
from sklearn.metrics import accuracy_score
'''
def train_val_dataset(d, val_split=0.25):
    train_idx, val_idx = train_test_split(list(d.data.index), test_size=val_split)
    return Subset(d, train_idx), Subset(d, val_idx)
'''
def train_val_dataset(d, val_split=0.25):

    train_idx, val_idx = train_test_split(list(d.data.index), test_size=val_split)
   
    # associa ad ogni elemento del dataset il numero di parole, e ordina in base al numero di parole
    len_train = [(i, len(d.data[i].split(" "))) for i in train_idx]
    len_train = sorted(len_train, key=lambda el: el[1], reverse = True)

    #definisco indici ordinati da passare poi al dataloader
    train_idx = [el[0] for el in len_train]

    # stessa cosa per il test
    len_val = [(i, len(d.data[i].split(" "))) for i in val_idx]
    len_val = sorted(len_val, key=lambda el: el[1], reverse = True)

    val_idx = [el[0] for el in len_val]
    
    return Subset(d, train_idx), Subset(d, val_idx)
#'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
'''
The colab session crashes after having createad the pickle file, so we report the items to remove in this list
since the information gets lost when the session crashes.
'''
to_remove = [226, 371, 412, 1884, 1887, 3612, 4904, 5008, 5054, 5116, 5736, 5971, 6574, 8066, 8084, 8498, 8634, 8711, 9117, 9148, 9178, 9299, 9348, 9564, 9783, 9817, 10019, 10032, 10036, 10177]

ds = dataset.AmazonDataset() 
ds.load_dataset()
print(to_remove)
ds.filter(to_remove)

train_ds, test_ds = train_val_dataset(ds)

# Define parameters
batch_size = params.BATCH_SIZE
hidden_dim = 128
embedding_size = params.NUM_FEATURES  # lunghezza embedding (selezionato uno a caso perchè tanto tutti hanno la stessa dimensione)
dropout_rate = params.DROPOUT_RATE
lr = params.LR
epochs = params.NUM_EPOCHS
best_loss = float('inf')

[226, 371, 412, 1884, 1887, 3612, 4904, 5008, 5054, 5116, 5736, 5971, 6574, 8066, 8084, 8498, 8634, 8711, 9117, 9148, 9178, 9299, 9348, 9564, 9783, 9817, 10019, 10032, 10036, 10177]


In [10]:
print(ds.maximum_embedding_len)

None


In [11]:
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
  
  label_list, text_list, = [], []
  
  for i, (text, label) in enumerate(batch):
    label_list.append(label)
    #print(len(text))
    #print(type(text))
    text = torch.stack((text)) # converts list of tensors to tensor of tensors
    #text = torch.tensor(text, dtype=torch.float64)
    #processed_text = torch.tensor(text) #, dtype=torch.int64
    text_list.append(text)
  
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = pad_sequence(text_list, batch_first=True, padding_value=0) #.size()
  
  return text_list, label_list


In [12]:
batch = [([torch.randn(3), torch.randn(3)], 2), ([torch.randn(3), torch.randn(3),torch.randn(3)],1)]
text_list, label_list = collate_batch(batch)

In [ ]:
# valuta di mettere un bucket iterator per averer ecnesioni di lunghezze simili
train_loader = DataLoader(train_ds, batch_size=32, collate_fn=collate_batch, shuffle=True, num_workers=0)  # , collate_fn=None, pin_memory=False)

# BUild Dataloader
lstm_model = net.SentimentAnalysis(batch_size,
                                   hidden_dim,
                                   embedding_size,
                                   dropout_rate)  # modificato LSTM con SentimentAnalysis (nome rete)

# optimization algorithm
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

#10036 --> 664 > 500
#print(len(ds.data[500].split(" ")))
from torch.autograd import Variable
to_train = True
torch.autograd.set_detect_anomaly(True)
ce = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
# train and validate
if to_train:
    for epoch in range(epochs):
        # training
        epoch_loss = 0
        epoch_acc = 0
        for idxs, (batch, labels) in enumerate(train_loader):
            
            print(batch.size()) #32, 500, 3072 --> 
            
            #batch = torch.stack(batch)
            #batch = batch.permute(1, 0, 2)
            
            # optimizer.zero_grad()
            predictions = lstm_model(batch)
            
            labels = torch.Tensor([x-1 for x in labels.data.numpy()]) #mapping classes 1-5 in 0-4
            long_labels = labels.type(torch.LongTensor)
            loss1 = 0.5 * ce(predictions, long_labels)
            
            #print("Float Preds")
            float_preds = torch.argmax(softmax(predictions), 1)
            float_preds = float_preds.type(torch.FloatTensor)
            
            loss2 = 0.5 * mse(float_preds, labels)
            loss = loss1.clone() + loss2.clone()
            loss = Variable(loss, requires_grad = True)
            accuracy = accuracy_score(float_preds.data, long_labels)
            
            # perform backpropagation
            optimizer.zero_grad()
            loss.backward()
            #print("ciao")
            optimizer.step()

            epoch_loss = epoch_loss + loss  # .item()
            epoch_acc = epoch_acc + accuracy
            torch.cuda.empty_cache()
        
        train_loss, train_acc = epoch_loss / len(train_loader), epoch_acc / len(train_loader)

        # save best model
        if train_loss < best_loss:
            best_valid_loss = train_loss
            torch.save(lstm_model.state_dict(), 'saved_weights_BiLSTM.pt')

        print(f'\tEpoch; {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')




torch.Size([32, 268, 3072])
torch.Size([32, 150, 3072])
torch.Size([32, 301, 3072])
torch.Size([32, 245, 3072])
torch.Size([32, 190, 3072])
torch.Size([32, 257, 3072])
torch.Size([32, 238, 3072])
torch.Size([32, 218, 3072])
torch.Size([32, 353, 3072])
torch.Size([32, 201, 3072])
torch.Size([32, 236, 3072])
torch.Size([32, 223, 3072])
torch.Size([32, 390, 3072])
torch.Size([32, 226, 3072])
torch.Size([32, 274, 3072])
torch.Size([32, 248, 3072])
torch.Size([32, 209, 3072])
torch.Size([32, 461, 3072])
torch.Size([32, 442, 3072])
torch.Size([32, 116, 3072])
torch.Size([32, 208, 3072])
torch.Size([32, 385, 3072])
torch.Size([32, 207, 3072])
torch.Size([32, 182, 3072])
torch.Size([32, 218, 3072])


In [ ]:
# load weights and make predictions
lstm_model.load_state_dict(torch.load("saved_weights_BiLSTM.pt"))
epoch_loss = 0
epoch_acc = 0

lstm_model.eval()

test_loader = DataLoader(test_ds, batch_size=32, shuffle=True, num_workers=0)  # , collate_fn=None, pin_memory=False)

with torch.no_grad():
    for idx, batch in test_loader:
        text, text_lengths = batch.text

        predictions = lstm_model(batch).squeeze(1)

        loss = 0.5 * ce(predictions, batch.labels) + 0.5 * mse(predictions, batch.labels)

        winners = predictions.argmax(dim=1)
        corrects = (winners == batch.labels)
        accuracy = corrects.sum().float() / float(batch.labels.size(0))

        epoch_loss += loss.item()
        epoch_acc += accuracy.item()

test_loss, test_acc = epoch_loss / len(test_iterator), epoch_acc / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')